# Extracting HRV Parameters for ECG Signals 

In this notebook, the focus is on how to extract some parameters from ECG, commonly used for heart rate variability analysis (HRV).

[Source](https://biosignalsplux.com/learn/notebooks/Categories/Extract/hrv_parameters_rev.php)

In [2]:
# Import libraries
import biosignalsnotebooks as bsnb
import pandas as pd
import rndSignal
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

In [3]:
sr = 125 # sampling rate
# CSV file for patients 1 and 5 from the BIDMC PPG and Respiration Dataset
# Source: physionet.org/content/bidmc/1.0.0/
# Focus on II column for ECG
df1 = pd.read_csv("/root/rndSignal/data/bidmc_01_Signals.csv")
df2 = pd.read_csv("/root/rndSignal/data/bidmc_05_Signals.csv")

# Get the first 30 seconds
# Minimum duration to ensure some functions will not run into errors
raw_p1 = df1[0:3750]

# Get the first 5 minutes
raw_p2 = df2[0:37500]

## Filtering of Raw Data

In [4]:
# Filtering the raw ECG signal data for patient 1
filtered_p1 = rndSignal.filter_ecg(sig = raw_p1[" II"], fs = sr)

In [5]:
# Filtering the raw ECG signal data for patient 2
filtered_p2 = rndSignal.filter_ecg(sig = raw_p2[" II"], fs = sr)

In [6]:
# Comparing raw vs. filtered data for patient 1
# Plot for patient 2 not shown due to longer duration -> more condensed plot
# "_ =" suppresses the output of the function when plotting
_ = rndSignal.filter_ecg(sig = raw_p1[" II"], fs = sr, display_signal = True)

## Signal Quality Assessment

For both patients, we can say that the noise influence was low on the overall signal.

In [7]:
# Computing signal-to-noise ratio (SNR) for patient 1
rndSignal.snr(sig = filtered_p1, sr = sr)

Amplitude of ECG signal: 1.352365394027691 mv
Amplitude of noise signal: 0.06408266725740532 mv
SNR for ECG signal: 26.48706938528296 dB


In [8]:
# Computing signal-to-noise ratio (SNR) for patient 2
rndSignal.snr(sig = filtered_p2, sr = sr)

Amplitude of ECG signal: 0.7698639716714306 mv
Amplitude of noise signal: 0.23969452890380727 mv
SNR for ECG signal: 10.135117492161854 dB


## Finding R Peaks

The typical heartbeat in an ECG signal is composed by various waves that correspond to specific events in the heart. 
Specifically, there are the P, Q, R, S and T waves that occur in the heart during the contraction and relaxation of the atria along with the contraction and relaxation of the ventricles. Being the R peak the most noticeable wave of each heartbeat, algorithms for heartbeat detection usually focus on finding these structures. In the plot below, [Neurokit's algorithm](https://neurokit2.readthedocs.io/en/latest/functions.html#neurokit2.ecg.ecg_findpeaks) is used to rapidly and accurately detect R peaks.

In [9]:
# Finding R peaks for patient 1
# Plot for patient 2 not shown due to longer duration -> more condensed plot
rndSignal.ecg_plot(sig = filtered_p1, sr = sr, plot_peaks = True)

## Generation of Tachogram

The **tachogram** corresponds to the derivative of the signal, i.e., to the interval duration of peaks over time, in order to get the **variation** of the interval duration of peaks over time (or the variation of the heartbeats period over time). It defines the fundamental structure from where all parameters will be extracted.

The next figures illustrate the tachogram and the corresponding ECG signal that originated it. The areas of the plots with different colors correspond to different heartbeats and indicate the interval considered to calculate the duration of each point in the tachogram.

In [10]:
# Plotting the tachogram for patient 1
rndSignal.ecg_plot(sig = filtered_p1, sr = sr, plot_tachogram = True)

In [11]:
# Plotting the tachogram for patient 2
rndSignal.ecg_plot(sig = filtered_p2, sr = sr, plot_tachogram = True)

## Removal of ectopic beats

An **ectopic beat** refers to a cardiac cycle that differs in at least 20% of the duration of the previous one and needs to be removed. The plots below show the comparison between the tachograms obtained before and after ectopic beat removal. For both plots, we can conclude that there are **no ectopic beats in the present acquisition**.

In [12]:
# Plotting the removal of ectopic beats for patient 1
rndSignal.ecg_plot(sig = filtered_p1, sr = sr, plot_ectopic_beats = True)

In [13]:
# Plotting the removal of ectopic beats for patient 2
rndSignal.ecg_plot(sig = filtered_p2, sr = sr, plot_ectopic_beats = True)

## Extraction of Parameters

This procedure can be done automatically using the **hrv_parameters** function.

In [14]:
# Extracting HRV parameters for patient 1
dictParameters = rndSignal.hrv_parameters(filtered_p1, sr, signal = True)
print(dictParameters)

{'MaxRR': 0.6480000000000001, 'MinRR': 0.6239999999999997, 'AvgRR': 0.6384347826086956, 'MaxBPM': 96.15384615384622, 'MinBPM': 92.59259259259258, 'AvgBPM': 93.9798420049033, 'SDNN': 0.00568618529539889, 'SD1': 0.0053274041115632794, 'SD2': 0.006023634439382903, 'SD1/SD2': 0.884416902316046, 'NN20': 0, 'pNN20': 0, 'NN50': 0, 'pNN50': 0, 'ULF_Power': 0.0, 'VLF_Power': 0.0, 'LF_Power': 0.0, 'HF_Power': 1e-05, 'LF_HF_Ratio': 0.0, 'Total_Power': 1e-05}


In [15]:
# Extracting HRV parameters for patient 2
dictParameters = rndSignal.hrv_parameters(filtered_p2, sr, signal = True)
print(dictParameters)

{'MaxRR': 0.632000000000005, 'MinRR': 0.5760000000000218, 'AvgRR': 0.6115828220858895, 'MaxBPM': 104.1666666666627, 'MinBPM': 94.93670886075874, 'AvgBPM': 98.10609100393226, 'SDNN': 0.007699214591649453, 'SD1': 0.005140631783712966, 'SD2': 0.009598422553774955, 'SD1/SD2': 0.5355704809735857, 'NN20': 0, 'pNN20': 0, 'NN50': 0, 'pNN50': 0, 'ULF_Power': 0.0, 'VLF_Power': 1e-05, 'LF_Power': 1e-05, 'HF_Power': 0.0, 'LF_HF_Ratio': inf, 'Total_Power': 2e-05}


### Time Parameters

In [16]:
# Plotting time parameters for patient 1
rndSignal.ecg_plot(sig = filtered_p1, sr = sr, plot_time = True)

In [17]:
# Plotting time parameters for patient 2
rndSignal.ecg_plot(sig = filtered_p2, sr = sr, plot_time = True)

### Poincaré Parameters

In [18]:
# Plotting poincare parameters for patient 1
rndSignal.ecg_plot(sig = filtered_p1, sr = sr, plot_poincare = True)

In [19]:
# Plotting poincare parameters for patient 2
rndSignal.ecg_plot(sig = filtered_p2, sr = sr, plot_poincare = True)

### Frequency Parameters

In [20]:
# Plotting frequency parameters for patient 1
rndSignal.ecg_plot(sig = filtered_p1, sr = sr, plot_frequency = True)

In [21]:
# Plotting frequency parameters for patient 2
rndSignal.ecg_plot(sig = filtered_p2, sr = sr, plot_frequency = True)